In [1]:
import pandas as pd
import re
import nltk

In [2]:
df=pd.read_csv("train.csv")
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [3]:
def cleanSentence(sentence):
    sentence=re.sub(r'#[A-Za-z0-9]\S+','', sentence)
    sentence=re.sub(r'http\S+', '', sentence)
    sentence=re.sub(r'@[A-Za-z0-9]\S+','',sentence)
    sentence=" ".join(word for word in sentence.split() if len(word)>3)
    return sentence.strip()

In [4]:
df['tweet']=df['tweet'].map(cleanSentence)
df.head

<bound method NDFrame.head of           id  label                                              tweet
0          1      0  when father dysfunctional selfish drags kids i...
1          2      0  thanks credit can't cause they don't offer whe...
2          3      0                                bihday your majesty
3          4      0  love take with time urð±!!! ðððð ...
4          5      0                                factsguide: society
...      ...    ...                                                ...
31957  31958      0  that youuu?ððððððððð...
31958  31959      0  nina turner airwaves trying wrap herself mantl...
31959  31960      0                listening songs monday morning work
31960  31961      1                                vandalised condemns
31961  31962      0                                       thank follow

[31962 rows x 3 columns]>

In [6]:
# Creating a dictionary of Words
words=[]
for sentences in df['tweet']:
    words.extend(re.split(r'\W+',sentences))
words[:10],len(words)

(['when',
  'father',
  'dysfunctional',
  'selfish',
  'drags',
  'kids',
  'into',
  'dysfunction',
  '',
  'thanks'],
 238049)

In [8]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
words=set([lemmatizer.lemmatize(word.lower()) for word in words])
len(words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sadan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


20380

In [9]:
# Creating Training Set
train_x=[]
words=list(words)
for sentence in df.tweet:
    word=set([word.lower() for word in sentence.split(" ")])
    temp=[]
    for w in words:
        temp.append(1 if w in word else 0)
    train_x.append(temp)
train_x[0].index(1)

1246

In [10]:
X=train_x
Y=df.label

In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [12]:
# Building Model
model=tf.keras.models.Sequential()
model.add(Dense(500,input_shape=(len(words),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

adam=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2)
model.fit(np.array(X_train),np.array(Y_train),epochs=5,batch_size=5)

Epoch 1/5
5114/5114 [==============================] - 601s 62ms/step - loss: 0.2610 - accuracy: 0.9311
Epoch 2/5
5114/5114 [==============================] - 330s 65ms/step - loss: 0.2508 - accuracy: 0.9328
Epoch 3/5
5114/5114 [==============================] - 337s 66ms/step - loss: 0.2618 - accuracy: 0.9332
Epoch 4/5
5114/5114 [==============================] - 326s 64ms/step - loss: 0.2570 - accuracy: 0.9334
Epoch 5/5
5114/5114 [==============================] - 323s 63ms/step - loss: 0.2507 - accuracy: 0.9338


In [13]:
# Testing the Model
count=0
for i in range(len(X_test)):
    prediction=model.predict(np.array([X_test[i]]))[0][0]
    if prediction>0.5 and np.array(Y_test)[i]==1:
        count+=1
    elif prediction<=0.5 and np.array(Y_test)[i]==0:
        count+=1
        
count/len(X_test)

0.9347724073205068